In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
# print(X.shape)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.NewtonRaphson(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.1, line_search_method='backtrack', line_search_cond='armijo')
# opt = pytorch_soom.NewtonRaphson(model.parameters(), lr=1, model=model, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='wolfe')
# opt = pytorch_soom.NewtonRaphson(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='strong-wolfe')
# opt = pytorch_soom.NewtonRaphson(model.parameters(), lr=1, model=model, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack', line_search_cond='goldstein')

all_loss = {}
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch+1] += loss
    all_loss[epoch+1] /= len(data_loader)
    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

epoch:  0, loss: 0.0635034441947937
epoch:  1, loss: 0.06324025243520737
epoch:  2, loss: 0.06291600316762924
epoch:  3, loss: 0.06277631223201752
epoch:  4, loss: 0.049927011132240295
epoch:  5, loss: 0.04730471968650818
epoch:  6, loss: 0.04482923075556755
epoch:  7, loss: 0.043098460882902145
epoch:  8, loss: 0.04071572795510292
epoch:  9, loss: 0.038298871368169785
epoch:  10, loss: 0.03662531077861786
epoch:  11, loss: 0.035006023943424225
epoch:  12, loss: 0.033718597143888474
epoch:  13, loss: 0.03237617015838623
epoch:  14, loss: 0.03126192092895508
epoch:  15, loss: 0.03010471537709236
epoch:  16, loss: 0.02903130277991295
epoch:  17, loss: 0.02802630327641964
epoch:  18, loss: 0.02710619382560253
epoch:  19, loss: 0.025034114718437195
epoch:  20, loss: 0.020562756806612015
epoch:  21, loss: 0.020029926672577858
epoch:  22, loss: 0.017945973202586174
epoch:  23, loss: 0.016117174178361893
epoch:  24, loss: 0.014889311045408249
epoch:  25, loss: 0.013658580370247364
epoch:  26,